# Comparables

## Team Members


| Name | EID |
| --- | --- |
| Akshat Johari | aj32864 |
| Aritra Chowdhury  | ac79277 |
| Brandt Green | bwg537 |

### Assumptions/Notes:

  
### Data Sources (All from WRDS)
 * [Stock Data (Pricing)](https://wrds-www.wharton.upenn.edu/pages/get-data/compustat-capital-iq-standard-poors/compustat/north-america-daily/security-daily/)
 * [Stock Data (Fundamentals)](https://wrds-www.wharton.upenn.edu/pages/get-data/compustat-capital-iq-standard-poors/compustat/north-america-daily/fundamentals-quarterly/)
 * [S&P 500 Data](https://wrds-www.wharton.upenn.edu/pages/get-data/center-research-security-prices-crsp/annual-update/index-sp-500-indexes/portfolios-on-sp-500/)
 * 

## Suggested Interface:
#### The best way to interact with this notebook, is to just click "run all cells", then scroll to the very bottom and play with the dashboard.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns; sns.set_theme()
import statsmodels.api as sm
from statsmodels.formula.api import ols
import datetime
import wrds
import warnings
import ipywidgets as widgets
from IPython.display import display
from ipywidgets import GridspecLayout
from ipywidgets import Layout

warnings.filterwarnings("ignore")

pd.set_option('display.float_format', lambda x: '%.5f' % x)

In [2]:
# Establish connection to WRDS

db = wrds.Connection(wrds_username='bgreen41')

Loading library list...
Done


## Helper Functions

In [3]:
def get_de_ratios(tickers:list, start_date:str, end_date:str) -> dict:
    """
    Returns the D/E Ratios of the Ticker and the Comps on the latest date entered by the user.
    
    Arguments: 
        ticker: Ticker
        comps: List of Comparables
        start_date: Start Date as entered by the user
        end_date: End Date as entered by the user
    
    Returns: 
        Comps and their D/E Ratios (Dictionary)
    
    """

    tickers = "'" + "','".join(tickers) + "'"
    query_string = f"""SELECT datadate, tic, dlttq, dlcq, icaptq FROM comp.fundq where tic IN ({tickers}) and datadate >= '{start_date}' and datadate <= '{end_date}' order by datadate desc"""
    de_data = db.raw_sql(query_string)
    
    # Calculate D/E Ratio for Ticker and Comps
    de_data['DE Ratio'] = (de_data['dlttq'] + de_data['dlcq'])/de_data['icaptq']
    
    de_ratios = de_data.sort_values(by = 'datadate').groupby('tic').last()['DE Ratio'].to_dict()
    
    return de_ratios

    

## Data Extraction

### [Market Data: S&P 500](https://wrds-www.wharton.upenn.edu/pages/get-data/center-research-security-prices-crsp/annual-update/index-sp-500-indexes/index-file-on-sp-500/)

In [4]:
def get_market_data(start_date, end_date) -> pd.DataFrame:
    query_string = f"""SELECT * FROM crsp.dsp500 where caldt >= '{start_date}' and caldt <= '{end_date}'"""
    market_data = db.raw_sql(query_string)
    market_data = market_data[['caldt','vwretd']] # vwretd is the total return on the index per period. This is given as a decimal value already
    market_data.columns = ['date','market_return']
    market_data['date'] = pd.to_datetime(market_data['date'])
    market_data.set_index('date', inplace = True)
    market_data['year'] = market_data.index.year
    market_data['month'] = market_data.index.month
    market_data['week'] = market_data.index.week
    # market_data = market_data[1:]
    return market_data

def get_stock_data(tickers:list, start_date:str, end_date:str) -> pd.DataFrame:
# Below line is kinda ugly, but works. Just converts the ticker list into the correct string format to use in the query.
    tickers_string = ''.join([f"'{ticker}'," for ticker in tickers])[:-1]

    query_string = f"""SELECT * FROM comp.secd where tic IN({tickers_string}) and datadate >= '{start_date}' and datadate <= '{end_date}' order by datadate"""
    stock_data = db.raw_sql(query_string)

    # If you get back an empty dataframe, it means no data was found.
    if len(stock_data) < 1:
        return None

    stock_data = stock_data[['datadate','tic','conm','ajexdi','prccd','trfd','cshoc']]
    stock_data.columns = ['date','ticker','name','adj_factor','price','stock_return_factor','shares_outstanding']
    stock_data['stock_return_factor'] = stock_data['stock_return_factor'].apply(lambda x: 1 if x is None else x)
    stock_data['price_factored'] = (stock_data['price'] / stock_data['adj_factor']) * stock_data['stock_return_factor']
    stock_data['market_cap'] = stock_data['price'] * stock_data['shares_outstanding']
    stock_data['stock_return'] = stock_data['price_factored'].pct_change()
    stock_data = stock_data[['date', 'stock_return','ticker','market_cap']][1:]
    stock_data['date'] = pd.to_datetime(stock_data['date'])
    stock_data.set_index('date', inplace = True)
    stock_data['year'] = stock_data.index.year
    stock_data['month'] = stock_data.index.month
    stock_data['week'] = stock_data.index.week

    return stock_data

#### Stock class below is where the bulk of the processing occurs.

In [5]:
class Stock:

    def __init__(self,ticker:str, start_date:datetime.date, end_date:datetime.date):
        self.ticker:str = str(ticker).upper()
        self.start_date:datetime.date = start_date
        self.end_date:datetime.date = end_date 

        self.beta_historical:float = None
        self.beta_comps:float = None
        self.debt_beta:float = None
        self.tax_rate:float = None
        self.de_ratio:float = None
        self.cost_of_equity:float = None
        self.cap_assumption:str =  None
        self.stock_data:pd.DataFrame = None
        self.comps_list:list = None
        self.comps_dict:dict = None

        self.equity_cost_scl:float = None
        self.equity_cost_comps:float = None

    

    def load_stock_data(self) -> pd.DataFrame:
        """Load the stock data as a variable in this isntance."""
        self.stock_data:pd.DataFrame =  get_stock_data([self.ticker], start_date, end_date)
        return self.stock_data


    def get_market_cap(self, date=None)->float:
        if self.stock_data is None:
            self.load_stock_data()
        if date is None:
            date = self.end_date

        stock_data_filtered = self.stock_data[self.stock_data.index.date <= date]
        market_cap = stock_data_filtered['market_cap'][-1]
        return market_cap

    def find_comps(self, comp_num=5) ->list:

        """Returns a particular number of comps for the ticker specified"""
            
        # Get Market Cap of Ticker
        mktcap = self.get_market_cap()
        
        # Get SIC of Ticker
        query_string = f"""SELECT siccd FROM crsp.stocknames WHERE ticker = '{self.ticker}' LIMIT 15"""
        sic_id = db.raw_sql(query_string)
        sic = int(sic_id.loc[0])

        # Get Companies in the same Sector
        query_string = f"""SELECT DISTINCT comnam, ticker FROM crsp.stocknames WHERE siccd = '{sic}'"""
        comp_names = db.raw_sql(query_string)
        comp_names.dropna(inplace = True)

        ticker_wrds = "'" + "','".join(list(comp_names['ticker'])) + "'"
        
        # Get Market Cap for Companies fetched above
        query_string = f"""SELECT tic, datadate, prccd, cshoc FROM comp.secd where tic in ({ticker_wrds}) and datadate >= '{self.start_date}' and datadate <= '{self.end_date}' order by datadate"""
        comp_data = db.raw_sql(query_string)
        comp_data['MarketCap'] = comp_data['prccd'] * comp_data['cshoc']
        comp_data = comp_data.sort_values('datadate')
        comp_cleaned = comp_data.groupby('tic').last().reset_index()[['tic', 'MarketCap']]
        comp_cleaned['Difference'] = abs(comp_cleaned['MarketCap'] - mktcap)
        comp_cleaned = comp_cleaned.sort_values('Difference')
        comp_cleaned = comp_cleaned.merge(comp_names, right_on = 'ticker', left_on = 'tic')
        
        try:
            comp_cleaned.drop(comp_cleaned.index[comp_cleaned['tic'] == self.ticker], inplace=True)
        except:
            pass

        comp_cleaned.drop_duplicates(subset = ['tic'], inplace = True)
        
        # Get the required Number of Comps
        valids = []
        for i in comp_cleaned['tic']:
            if comp_checker(i, self.start_date, self.end_date):
                valids.append(i)
            if len(valids) == comp_num:
                break

        self.comps_list:list = valids
            
        return valids

    
    def get_historical_equity_beta(self, market_data:pd.DataFrame=None, risk_free:float=0.0, frequency:str='Weekly') -> float:
        """
        Returns the Historical Equity Beta through SCL Regression. 
        
        Arguments: 
            market_data: The S&P 500 (Market Proxy) returns as a DataFrame
            risk_free: The Risk-free Rate as entered by the user
            frequency: The Frequency of Returns as entered by the user
        
        Returns: 
            The Historical Equity Beta of the Ticker (Scalar)
        
        """
        # First, make sure you've got the data loaded
        if market_data is None:
            market_data = get_market_data(self.start_date,self.end_date)
        if self.stock_data is None:
            self.load_stock_data()

        # Group By Frequency Setting according to Input Frequency
        if frequency == 'Yearly':
            group_list = ['year']
        elif frequency == 'Monthly':
            group_list = ['year', 'month']
        elif frequency == 'Weekly':
            group_list = ['year', 'week']
        else:
            group_list = ['date']
        
        # Function to Calculative Return according to Input Frequency
        def cum_return(series):
            return (series + 1).prod() - 1
        
        # Restructure DataFrames with Returns according to Input Frequency
        stock_data = self.stock_data.groupby(group_list).agg({'stock_return': cum_return})
        market_data = market_data.groupby(group_list).agg({'market_return': cum_return})
        
        # Merge with Market Data to get both returns in the same DataFrame
        all_data = stock_data.merge(market_data, left_index = True, right_index = True)
        all_data['stock_risk_premium'] = all_data['stock_return'] - risk_free
        all_data['market_risk_premium'] = all_data['market_return'] - risk_free
        
        # Model SCL Regression and Calculate Equity Beta
        model = ols('stock_risk_premium ~ market_risk_premium', data = all_data)
        results = model.fit()
        self.beta_historical = results.params[1]

        return self.beta_historical

    def get_de_ratio(self):
        ratios = get_de_ratios([self.ticker],self.start_date, self.end_date)
        self.de_ratio = ratios[self.ticker]
        return self.de_ratio

    
    def calculate_cost_of_equity(self, exp_market_return:float, risk_free:float=0.0,) -> float:
        """
        This function calculates the Cost of Equity based on the Beta, Risk Free Rate and the Expected Market Return.
        
        Arguments: 
            beta: The Beta of the Ticker as computed by other functions
            risk_free: The Risk-free Rate as entered by the user
            exp_market_return: The Expected Market Return as entered by the user
        
        Returns: 
            The Relevered Beta of the Ticker (Scalar)

        """
        if self.beta_historical is not None:
            self.equity_cost_scl = risk_free + self.beta_historical * (exp_market_return - risk_free)
        if self.beta_comps is not None:
            self.equity_cost_comps = risk_free + self.beta_comps * (exp_market_return - risk_free)
        


    def calculate_unlevered_beta_historical(self):

        if self.cap_assumption == 'Constant Debt':
            unlevered_beta = self.beta_historical / (1 + (1 - self.tax_rate) * self.de_ratio)
        else:
            unlevered_beta = (self.beta_historical + (self.debt_beta * self.de_ratio)) / (1 + self.de_ratio)

        return unlevered_beta

    def get_equity_beta_comps(self, market_data:pd.DataFrame=None, risk_free:float=0, frequency:str='Weekly'):
        """
        Returns the Relevered Beta of the Ticker based on the Betas of Comps.
        
        Arguments: 
            ticker: Ticker
            comps: List of Comparables
            start_date: Start Date as entered by the user
            end_date: End Date as entered by the user
            market_data: The S&P 500 (Market Proxy) returns as a DataFrame
            risk_free: The Risk-free Rate as entered by the user
            frequency: The Frequency of Returns as entered by the user
            tax_rate: The Effective Tax Rate
            debt_beta: The Debt Beta as entered by the user
        
        Returns: 
            The Relevered Beta of the Ticker (Scalar)
        
        """

        if self.comps_dict is None:
            raise Exception('Hey buddy, you need to create the comps_dict before trying to find the equity beta using the comps.') 

        # First, make sure you've got the data loaded
        if market_data is None:
            market_data = get_market_data(self.start_date,self.end_date)
        if self.stock_data is None:
            self.load_stock_data()
        
        self.get_de_ratio()

        
        # Unlevered Beta for Comps
        unlevered_betas = {}
        for ticker, comp in self.comps_dict.items():
            
            # Get D/E Ratios of Ticker and Comps
            comp.get_historical_equity_beta(market_data,risk_free,frequency)
            comp.get_de_ratio()

            # Calculate Unlevered Beta according to respective Capital Structure Assumption
            unlevered_beta = comp.calculate_unlevered_beta_historical()

            unlevered_betas.update({ticker: unlevered_beta})

        # Mean Unlevered Beta
        unlevered_beta = sum(unlevered_betas.values()) / len(unlevered_betas)

        # Calculate Relevered Beta according to Capital Structure Assumption
        if self.cap_assumption == 'Constant Debt':
            relevered_beta = unlevered_beta * (1 + (1 - self.tax_rate) * self.de_ratio)
        else:
            relevered_beta = (unlevered_beta * (1 + self.de_ratio)) - (self.debt_beta * self.de_ratio)
            
        self.beta_comps = relevered_beta
        return self.beta_comps

    
    def present_cost_of_equity_results(self) -> pd.DataFrame:
        """Spit out that dataframe of aggregated info."""

        data = {'Index': ['Beta', 'Cost of Equity'],
        'SCL Regression': [self.beta_historical, self.equity_cost_scl],
        'Comps Analysis': [self.beta_comps, self.equity_cost_comps]}

        df_result = pd.DataFrame(data).set_index('Index')

        return df_result

    def __repr__(self) -> str:
        return self.ticker
  


#### Return summary statistics:

In [6]:
def daily_var_1(series:pd.Series):
    """
    Calculate the the daily value at risk of the returns with a 1% threshold.
    """
    return np.percentile(series, .01)

def conditional_var_1(series:pd.Series):
    """
    Calculate the average loss of returns, given that the 1% threshold has been breached.
    """
    bottom_centile = series[series <= np.percentile(series, .01)]
    return bottom_centile.mean()

def max_draw_down(series:pd.Series):
    """
    What is the largest peak to trough percentage decline of the returns.
    """
    cumulative_returns = np.cumproduct((series + 1))
    highest_value = 1
    biggest_draw_down = 0
    for port_value in cumulative_returns:
        peak_to_trough_return = port_value/highest_value - 1
        if peak_to_trough_return < biggest_draw_down:
            biggest_draw_down = peak_to_trough_return
        if port_value > highest_value:
            highest_value = port_value

    return biggest_draw_down

def draw_down_days(series:pd.Series):
    """
    What is the largest peak to trough percentage decline of the returns.
    """
    cumulative_returns = np.cumproduct((series + 1))
    highest_value = 1
    biggest_draw_down = 0
    peak_date = series.index[0]
    draw_down_start_date = None # Initially the draw down start date will be the first date
    draw_down_end_date = None

    for date, port_value in zip(series.index, cumulative_returns):
        peak_to_trough_return = port_value/highest_value - 1
        if peak_to_trough_return < biggest_draw_down:
            biggest_draw_down = peak_to_trough_return
            draw_down_start_date = peak_date
            draw_down_end_date = date

        if port_value > highest_value: # Set the new highest portfolio value and draw_down_start_date
            highest_value = port_value
            peak_date = date

    days_of_drawdown = (draw_down_end_date - draw_down_start_date)
    return days_of_drawdown

def downside_std(series:pd.Series):
    """
    Standard deviation of negative returns only.
    """
    mn = series.mean()
    return np.sqrt(((series[series < 0] - mn)**2).sum() / len(series))

def proportion_positive_returns(series:pd.Series):
    """
    What percent of the daily portfolio returns are postive?
    """
    return len(series[series>0])/len(series)

def annualized_return(series:pd.Series):
    """
    Calculate annualized returns
    """
    days = len(series)
    
    return ((np.cumproduct(series + 1)**(252/days))[-1] - 1)

def calculate_portfolio_statistics(df_returns:pd.DataFrame) ->pd.DataFrame:
    """
    This function is here to just aggregate all the relevant info and make it presentable.
    """
    from scipy.stats import kurtosis

    # Groupby ticker and get the statistics below.
    summary_stats = df_returns.groupby(['ticker']).aggregate(['mean', 'std', 'skew',
     kurtosis, daily_var_1, conditional_var_1, max_draw_down, draw_down_days, downside_std, 
                                                              proportion_positive_returns, annualized_return])

    # Reset Column Indexes post groupby
    summary_stats.columns = summary_stats.columns.get_level_values(1)

    # Create a few new performance metrics based on the statistics above:
    summary_stats['Annualized_Return'] = summary_stats['annualized_return']
    summary_stats['Annualized_Std'] = summary_stats['std'] * 252**.5
    summary_stats['Sharpe_Ratio'] = summary_stats['Annualized_Return']/summary_stats['Annualized_Std']
    summary_stats['Sortino_Ratio'] = summary_stats['Annualized_Return']/summary_stats['downside_std']

    # Make the df pretty:
    summary_stats = summary_stats.drop(columns=['mean','std','downside_std','annualized_return'])
    summary_stats.columns = [column.title() for column in summary_stats.columns]# Rearrange columns
    
    # Rearrange columns
    # summary_stats = summary_stats[['Annualized_Return','Annualized_Std','Sharpe_Ratio','Sortino_Ratio','Skew','Kurtosis',
    # 'Daily_Var_1','Conditional_Var_1','Max_Draw_Down','Draw_Down_Days','Proportion_Positive_Returns']] 

    summary_stats = summary_stats[['Annualized_Return','Annualized_Std','Sharpe_Ratio','Skew','Kurtosis',
    'Daily_Var_1','Conditional_Var_1','Max_Draw_Down','Draw_Down_Days','Proportion_Positive_Returns']] 
    
    # Rename columns
    summary_stats.rename(columns = {'Annualized_Return':'Annualized Return',
                                    'Annualized_Std': 'Annualized Standard Deviation',
                                    'Sharpe_Ratio': 'Sharpe Ratio',
                                    # 'Sortino_Ratio': 'Sortino Ratio',
                                    'Daily_Var_1': 'Daily Value at Risk (1% Threshold)',
                                    'Conditional_Var_1': 'Conditional Value at Risk (1% Threshold)',
                                    'Max_Draw_Down': 'Maximum Drawdown',
                                    'Draw_Down_Days': 'Maximum Drawdown Days',
                                    'Proportion_Positive_Returns': 'Proportion of Positive Returns'}, inplace = True)
    
    return summary_stats.T

In [7]:
def merge_stock_dfs_flat(stocks_list:list) -> pd.DataFrame:
    combined_df = pd.DataFrame()
    for stock in stocks_list:
        combined_df = combined_df.append(stock.stock_data)
    combined_df = combined_df.reset_index()[['date','stock_return','ticker']]
    combined_df = combined_df.pivot_table(index='date',columns='ticker')

    # # Pivot makes our column names multi-indexed. We want the sub-index
    combined_df.columns = combined_df.columns.get_level_values(1) 
    # combined_df = combined_df.rename(columns = {'':'date'}).set_index('date') 
    return combined_df

def merge_stock_dfs_stacked(stocks_list:list) -> pd.DataFrame:
    combined_df = pd.DataFrame()
    for stock in stocks_list:
        combined_df = combined_df.append(stock.stock_data)

    return combined_df[['stock_return','ticker']].copy()

#### Validation Functions

In [8]:
def start_before_end(start_date:datetime.date,end_date:datetime.date) ->bool:
    return start_date < end_date

def date_in_df(date:datetime.date, df:pd.DataFrame) -> bool:
    # Need to access the date portion of the pandas datetime variable
    all_dates = df.index.date 
    first_date_date = all_dates[0]
    last_data_date = all_dates[-1]

    if date >= first_date_date and date <= last_data_date:
        return True
    else:
        return False

def get_fundq_df(tickers:list,start_date,end_date):
    tickers = "'" + "','".join(tickers) + "'"
    query_string = f"""SELECT datadate, tic, dlttq, dlcq, icaptq FROM comp.fundq where tic IN ({tickers}) and datadate >= '{start_date}' and datadate <= '{end_date}' order by datadate desc"""
    df = db.raw_sql(query_string)
    if len(df) == 0:
        return None

    df = df.set_index(keys='datadate')
    df.index = pd.to_datetime(df.index)
    df = df.sort_index()
    return df

def comp_checker(ticker:str, start_date, end_date)->bool:
    """lolol"""
    start_date_buffered, end_date_buffered = start_date - datetime.timedelta(days=10), end_date + datetime.timedelta(days=10)
    start_date_buffered_super, end_date_buffered_super = start_date - datetime.timedelta(days=35), end_date + datetime.timedelta(days=35)
    ## SEC db
    stock_df = get_stock_data([ticker],start_date_buffered,end_date_buffered)
    # fundq db
    fund_q_db = get_fundq_df([ticker],start_date_buffered_super,end_date_buffered_super)
    if fund_q_db is None:
        return False
        
    if date_in_df(start_date,stock_df) and date_in_df(end_date,stock_df):
        if date_in_df(start_date,fund_q_db) and date_in_df(end_date,fund_q_db):
            return True
    return False


Below two functions are helpful for grabbing info from the dashboard and creating the appropriate variables

In [9]:
def extract_comp_inputs() -> dict:
    comp_dict = {}

    for comp_box in comp_grid.children:
        comp_buttons = comp_box.children
        if comp_buttons[0].value != '':
            ticker = comp_buttons[0].value.upper()
            cap_assumption = comp_buttons[1].value
            comp_debt_beta = comp_buttons[2].value

            stock = Stock(ticker=ticker,start_date=start_date, end_date=end_date)
            stock.cap_assumption = cap_assumption
            stock.tax_rate = tax_rate
            if cap_assumption == 'Constant Debt':
                stock.debt_beta = 0
            else:
                stock.debt_beta = comp_debt_beta

            comp_dict[ticker] = stock
    return comp_dict

def create_stock_from_inputs() -> Stock:
    ticker_stock = Stock(ticker,start_date,end_date)
    ticker_stock.cap_assumption = ticker_cap_assumption
    ticker_stock.tax_rate = tax_rate
    if comp_cap_assumption == 'Constant Debt':
        ticker_stock.debt_beta = 0
    else:
        ticker_stock.debt_beta = ticker_debt_beta
    return ticker_stock


Dash board utils:

In [10]:
def create_comp_grid():

    def create_comp_box(comp_num:int):
        x1 = widgets.Text(description=f'Comp {comp_num}:',layout=Layout(width='200px'), style= {'description_width': 'initial'})
        x2 = widgets.RadioButtons(options=['Dynamic', 'Constant Debt'],value='Dynamic',layout=Layout(margin='20px 0 0 0'))
        debt_beta_input = widgets.FloatText(value=.3,step=0.01,description='Debt-Beta:',style= {'description_width': 'initial'},layout=Layout(margin='20px 0 0 0',width='200px'))
        return widgets.VBox([x1,x2,debt_beta_input],layout=Layout())
        
    grid = GridspecLayout(2, 3,height='450px')

    comp_count = 1
    for row in range(2):
        for column in range(3):
            grid[row,column] = create_comp_box(comp_count)
            comp_count += 1

    return grid

#### Creating buttons and such for dash:

In [11]:
ticker_input = widgets.Text(value='MSFT',description='Ticker:')
start_date_input = widgets.DatePicker(description='Start Date',value = datetime.date(2015,12,31))
end_date_input = widgets.DatePicker(description='End Date', value = datetime.date(2020,12,31))

freq_input = widgets.ToggleButtons(options=['Daily', 'Weekly', 'Monthly', 'Yearly'], description='Data Frequency:',
     value = 'Weekly', button_style='info', style= {'description_width': 'initial'})


ticker_cap_input = widgets.RadioButtons(options=['Dynamic', 'Constant Debt'],value='Dynamic',description='Cap Structure:',
    style= {'description_width': 'initial'})

debt_beta_input = widgets.FloatText(value=.3,step=0.05,description='Debt-Beta:',style= {'description_width': 'initial'})



comp_method_input = widgets.ToggleButtons(options=['Manual', 'Automatic'], description='Comp Determination:',
     value = 'Automatic', button_style='info', style= {'description_width': 'initial'})

comp_count_input = widgets.IntText(value=5,step=1,min=1,description='Comp Number (If Automatic)',style= {'description_width': 'initial'})
comp_debt_beta_input = widgets.FloatText(value=.3,step=0.05,description='Debt-Beta (If Automatic)',style= {'description_width': 'initial'})
comp_cap_input = widgets.RadioButtons(options=['Dynamic', 'Constant Debt'],value='Dynamic',description='Cap Structure (If Automatic)',
    style= {'description_width': 'initial'})

comp_grid = create_comp_grid()


risk_free_input = widgets.FloatText(value=.01, step=0.01, description='Risk-Free:')
market_return_input = widgets.FloatText(value=.07,step=0.01,description='Expected Market Return',style= {'description_width': 'initial'})
tax_input  = widgets.FloatText(value=.21,step=0.01,description='Marginal Tax Rate',style= {'description_width': 'initial'})


main_header = widgets.HTML(value="""<h1 style="color:darkblue">Input Program Parameters & Assumptions<h1><hr>""")
market_header = widgets.HTML(value="<h2>Capital Market Assumptions & Other:<h2>")


# Main Execution
Running the below cell will create the dashboard that controls all of the magic.

In [12]:
display(main_header)

display(widgets.HTML("""<h2>Stock Inputs:</h2>"""))
display(ticker_input)
display(start_date_input)
display(end_date_input)
display(freq_input)
display(ticker_cap_input)
display(debt_beta_input)

display(widgets.HTML("""<h2>Comp Inputs:</h2>"""))
# display(widgets.HTML("""How would you like to determine the comps?"""))
display(comp_method_input)
display(comp_count_input)
display(comp_cap_input)
display(comp_debt_beta_input)
display(widgets.HTML("<br>"))
display(comp_grid)


display(market_header)
display(widgets.HBox([tax_input,risk_free_input,market_return_input]))



# ------------------------- Validation---------------------
display(widgets.HTML("<h2>Validation<h2>"))

validation_message = widgets.Output()

def create_error_message(error_text:str):
    return widgets.HTML(f"""<h3 style="color:Red">Error:</h3><ul><li>{error_text}</li></ul>""")


def validate_inputs(b):
    # We Want all the parameters set in here to be global so that we can access them outside of the function.
    global start_date; global end_date; global ticker; global stock_data_df;global start_date;
    global comps_dict; global risk_free; global tax_rate; global comp_number; global frequency;
    global exp_market_return; global ticker_debt_beta; global comp_debt_beta; global comp_cap_assumption;
    global ticker_cap_assumption; global stock;

    with validation_message:
        validation_message.clear_output()

        display(widgets.HTML("<h4>Validation in progress...</h4>"))

        # -----------Grab button values-------------
        # Yes, I know this looks trashy.
        ticker = str(ticker_input.value).upper() # Tickers should always be uppercase
        risk_free = risk_free_input.value; tax_rate = tax_input.value; comp_number = comp_count_input.value
        frequency = freq_input.value; exp_market_return = market_return_input.value
        start_date, end_date = start_date_input.value, end_date_input.value; comp_debt_beta = comp_debt_beta_input.value;
        ticker_debt_beta = debt_beta_input.value; ticker_cap_assumption = ticker_cap_input.value; comp_cap_assumption = comp_cap_input.value;

        # We need to add a buffer in the start and end dates so that we make sure we grab enough data from words for validation
        start_date_buffered, end_date_buffered = start_date - datetime.timedelta(days=10), end_date + datetime.timedelta(days=10)

        
        if not start_before_end(start_date,end_date):
            return display(create_error_message('Start date cannot be after end date you goon!!!'))
            
        # Grab the data for this stock and then validate!
        stock_data_df = get_stock_data([ticker], start_date_buffered, end_date_buffered)

        if stock_data_df is None:
            return display(create_error_message(f"Sorry, we could not find any data for '{ticker}' between '{start_date}' and '{end_date}'."))
        elif not date_in_df(start_date,stock_data_df):
            # print(stock_data_df)
            return display(create_error_message(f"Sorry, we do not have data for start date '{start_date}' for '{ticker}'. The earliest date we have is: '{stock_data_df.index[0]}'"))
        elif not date_in_df(end_date,stock_data_df): 
            return display(create_error_message(f"Sorry, we do not have data for start date '{end_date}' for '{ticker}'. The latest date we have is: '{stock_data_df.index[-1]}'"))

        # Yay, create the stock then.
        stock = create_stock_from_inputs()
        stock.load_stock_data() # This is just helpful to do now rather than later... I think. Tho it is super redundant.
        

        if comp_method_input.value == 'Manual':
            # Get a dictionary where the keys are tickers and the values are 'Stock' objects.
            comps_dict = extract_comp_inputs() 

            for comp in comps_dict.values():
                comp_stock_data = get_stock_data([comp.ticker], start_date_buffered, end_date_buffered)
                if comp_stock_data is None:
                    return display(create_error_message(f"Sorry, we could not find any data for '{comp.ticker}' between '{start_date}' and '{end_date}'."))
                elif not date_in_df(start_date,comp_stock_data):
                    # print(stock_data_df)
                    return display(create_error_message(f"Sorry, we do not have data for start date '{start_date}' for '{comp.ticker}'. The earliest date we have is: '{comp_stock_data.index[0]}'"))
                elif not date_in_df(end_date,comp_stock_data): 
                    return display(create_error_message(f"Sorry, we do not have data for start date '{end_date}' for '{comp.ticker}'. The latest date we have is: '{comp_stock_data.index[-1]}'"))        

                # If you pass all of the tests, load the comp with its appropriate inputs
                stock.comps_dict = comps_dict
                stock.comps_list = comps_dict.keys()

        else:
            comp_number = comp_count_input.value
            stock.find_comps(comp_num=comp_number) # Get comps automatically

            # Now, need to create the comps_dict and set each comp with the appropraite default values
            comps_dict = {}
            for comp_ticker in stock.comps_list:
                comp_stock = Stock(ticker=comp_ticker,start_date=start_date,end_date=end_date)
                comp_stock.tax_rate = tax_rate
                comp_stock.cap_assumption = comp_cap_assumption

                if comp_stock.cap_assumption == 'Constant Debt':
                    comp_stock.debt_beta = 0
                else:
                    comp_stock.debt_beta = comp_debt_beta

                comps_dict[comp_ticker] = comp_stock

            stock.comps_dict = comps_dict
              

        # If you make it here without hitting any errors then you are good to go!
        validation_message.clear_output()
        display(widgets.HTML(f"""<h2 style="color:Green">All Good!!</h2>"""))


validate_all = widgets.Button(description='Validate All Inputs',button_style='info')
display(validate_all, validation_message)
validate_all.on_click(validate_inputs)


#---------------------- Execute the code ------------------------

report_message = widgets.Output()

def master_executor():
    """One function to rule them all, one function to find them, one function to bring them all, and in the darkness bind them."""
    market_data = get_market_data(start_date, end_date)
    stock.get_historical_equity_beta(market_data,risk_free=risk_free, frequency='Daily')
    stock.get_equity_beta_comps(market_data=market_data, risk_free=risk_free, frequency='Daily')
    stock.calculate_cost_of_equity(exp_market_return=exp_market_return,risk_free=risk_free)

    report_message.clear_output()

    display(widgets.HTML("<h2>Beta & Cost of Capital</h2>"))

    return display(stock.present_cost_of_equity_results())

def plot_stock_returns(df_returns:pd.DataFrame):
    cum_return = (1+df_returns).cumprod()
    cum_return.plot(figsize = (17,5))
    plt.title(f'Cumulative Returns')
    plt.ylabel('Wealth Index')
    plt.show()


def execute_all_code(b):

    with report_message:
        report_message.clear_output()
        display(widgets.HTML("<h4>Processing...</h4>"))

        master_executor()

        all_stocks = [stock] + list(stock.comps_dict.values())
        df_summary_stats = calculate_portfolio_statistics(merge_stock_dfs_stacked(all_stocks))
        df_plotting = merge_stock_dfs_flat(all_stocks)

        display(widgets.HTML("<br>"))
        display(plot_stock_returns(df_plotting))
        display(widgets.HTML("<h2>Return Summary Statistics</h2>"))
        display(df_summary_stats)
        


report_btn = widgets.Button(description='Run Report', button_style='success')
display(report_btn, report_message)
report_btn.on_click(execute_all_code)


HTML(value='<h1 style="color:darkblue">Input Program Parameters & Assumptions<h1><hr>')

HTML(value='<h2>Stock Inputs:</h2>')

Text(value='MSFT', description='Ticker:')

DatePicker(value=datetime.date(2015, 12, 31), description='Start Date')

DatePicker(value=datetime.date(2020, 12, 31), description='End Date')

...

HTML(value='<h2>Validation<h2>')

Button(button_style='info', description='Validate All Inputs', style=ButtonStyle())

Output()

Button(button_style='success', description='Run Report', style=ButtonStyle())

Output()